# Models

* In the following, we'll compare a simple Markowitz Portfolio Optimization (Modern Portfolio Theory) to its Black-Litterman respective using views of a "well calibrated" portifolio manager.

In [54]:
import pytz
import scipy
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import scipy.stats as stat
import matplotlib.pyplot as plt

from src import dataset
from pathlib import Path
from datetime import datetime, timedelta

tz = pytz.timezone('Brazil/East')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

In [2]:
def get_rates(symbols, *, from_datetime, to_datetime, timeframe=mt5.MT5_TIMEFRAME_D1):
    mt5.MT5Initialize()
    mt5.MT5WaitForTerminal()
    full_list_rates = []
    for symbol in symbols:
        rates_for_symbol = pd.DataFrame(
            list(mt5.MT5CopyRatesRange(symbol, timeframe, from_datetime, to_datetime)), 
            columns=["time","open","low","high","close","tick_volume","spread","real_volume"])
        rates_for_symbol["symbol"] = symbol
        full_list_rates.append(rates_for_symbol)
    mt5.MT5Shutdown()
    return pd.concat(full_list_rates).set_index('time')


In [98]:
rates.min()

KeyError: 'index'

In [14]:
assets = ['ABEV3', 'FLRY3', 'MEAL3', 'CSNA3', 'JHSF3', 'GFSA3']

from_datetime = datetime(2017, 1, 1, tzinfo=tz)
to_datetime = datetime(2019, 12, 31, tzinfo=tz) 

rates = get_rates(assets, 
          from_datetime=from_datetime,
          to_datetime=to_datetime, 
          timeframe=mt5.MT5_TIMEFRAME_D1)

## MPT (Mean variance optimization)

In [4]:
close_dayly_prices = pd.DataFrame(
    dict((*rates.groupby("symbol").close, )))

In [5]:
def get_returns(df):
    return df\
           .diff()\
           .div(df.shift())

### Average returns

In [6]:
n_of_weekdays = len(rates.index.unique())

avg_returns = get_returns(close_dayly_prices)\
    .mean()\
    .multiply(n_of_weekdays)

### Covariance

In [7]:
Covariance = get_returns(close_dayly_prices)\
    .cov()\
    .multiply(n_of_weekdays)

### Maximize *sharpe ratio* (Minimize the variance and maximize the average returns)

In [8]:
def sharpe_ratio(portifolio_weights, 
                 expected_returns, 
                 returns_Covariance, 
                 risk_free_rate):
    portifolio_returns = portifolio_weights.dot(expected_returns)
    returns_variance =\
        np.sqrt(np.dot(portifolio_weights, 
                       np.dot(returns_Covariance, portifolio_weights.T)))
    return (portifolio_returns - risk_free_rate) / returns_variance

In [9]:
import scipy.optimize as sco

In [10]:
expected_returns = avg_returns # EXPLAIN
expected_Covariance = Covariance # EXPLAIN
risk_free_rate = 0.02 # EXPLAIN
n_assets = len(rates.groupby("symbol"))

initial_weigts_guess = np.array([1 / n_assets] * n_assets) # Equal weigts for each asset

cost_function_args = (expected_returns, expected_Covariance, risk_free_rate)
result = sco.minimize(
    lambda *args: -1.0 * sharpe_ratio(*args), # EXPLAIN
    x0 = initial_weigts_guess,
    args = cost_function_args,
    method = "SLSQP",
    bounds = tuple((0, 1) for _ in range(n_assets)),
    constraints = [{"type": "eq", "fun": lambda x: np.sum(x) - 1}],
)

weigts = dict(zip(assets, result['x']))
pd.DataFrame(weigts, index=[1])

,ABEV3,CSNA3,FLRY3,GFSA3,JHSF3,MEAL3
1,6.825331e-17,5.863422e-16,0.0,0.274417,0.420449,0.305135


# Backtest with zipline

In [11]:
%load_ext zipline

In [91]:
from src import dataset

path_to_save = Path("C:\Python36\Lib\site-packages\zipline\data\csvdata")
dataset.create_zipline_dataset(
    rates, 
    path_to_save=path_to_save)

c:\python36\lib\site-packages\zipline\extensions.py:21: UserWarning: Overwriting bundle with name 'custom-bundle'
  end_session=datetime(2019, 12, 31, tzinfo=tz)


NameError: name 'to_datetime' is not defined

In [90]:
!zipline ingest -b custom-bundle

Error: No bundle registered with the name 'custom-bundle'


In [16]:
%%zipline --start 2019-1-1 --end 2019-12-31 --capital-base 10000.0 -o buy_and_hold.pkl

from zipline.api import order_percent, symbols
from zipline.finance import commission
import numpy as np 
import pandas as pd

assets = ['ABEV3', 'FLRY3', 'MEAL3', 'CSNA3', 'JHSF3', 'GFSA3']

def initialize(context):
    context.set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    
    context.assets = symbols(*assets)
    context.n_assets = len(context.assets)
    context.has_position = False

def handle_data(context, data):
    if not context.has_position:
        for asset in context.assets:
            order_percent(asset, 1/context.n_assets)
        context.has_position = True

SymbolNotFound: Symbol 'ABEV3' was not found.

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [173]:
avg_returns = expected_returns.mean_historical_return(close_dayly_prices)
Covariance = risk_models.sample_cov(close_dayly_prices)
avg_returns

ABEV3    0.201305
CSNA3    0.614496
FLRY3    0.508252
GFSA3   -0.413223
JHSF3    1.449121
MEAL3    0.330418
dtype: float64

avg_returns

In [124]:
ef = EfficientFrontier(avg_returns, Covariance)
weights = ef.max_sharpe()
weights

{'ABEV3': 0.0,
 'CSNA3': 0.09713547284320327,
 'FLRY3': 0.21772011760060692,
 'GFSA3': 2.792650565020393e-15,
 'JHSF3': 0.5480970011507317,
 'MEAL3': 0.13704740840545682}